<a href="https://colab.research.google.com/github/jepilogo97/nlp/blob/main/nlp-with-lstm/nlp_with_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP con Transformers

##### Jean Pierre Londoño González
##### Mini-Proyecto de clasificación de texto con Transformers
##### 30AGO2025

En este notebook se implementa un clasificador de conversaciones cotidianas en español utilizando transformers. El dataset empleado corresponde a Everyday Conversations LLaMA 3.1 - 2k, disponible en Hugging Face, el cual contiene diálogos en español sobre diferentes temas. Para la preparación de los datos se realiza la tokenización empleando las utilidades de la librería Hugging Face Transformers.

#### Referencias
- Dataset: https://huggingface.co/datasets/HuggingFaceTB/everyday-conversations-llama3.1-2k

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la manipulación de datos y la construcción del modelo. Esto incluye NumPy y pandas para el manejo y análisis de datos; Hugging Face Datasets y Transformers para la carga de corpus y la tokenización; y PyTorch junto con PyTorch Lightning para definir, entrenar y evaluar el modelo de manera estructurada.

Además, se emplean torchmetrics y scikit-learn para calcular métricas de rendimiento como precisión, matrices de confusión y reportes de clasificación, mientras que Optuna permite la optimización automática de hiperparámetros. 

Finalmente, tqdm facilita el seguimiento del progreso de los procesos iterativos.

In [32]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

In [33]:
!wget -O requirements.txt https://raw.githubusercontent.com/jepilogo97/nlp/main/nlp-with-transformers/requirements.txt
!pip install -r requirements.txt

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [34]:
# Procesamiento de lenguaje natural y utilidades
import numpy as np  # Cálculo numérico y manejo de arreglos multidimensionales
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame

pd.set_option("display.max_rows", None)     # Todas las filas
pd.set_option("display.max_columns", None)  # Todas las columnas
pd.set_option("display.width", None)        # No cortar líneas

from datasets import load_dataset, concatenate_datasets  # Carga y combinación de datasets de Hugging Face
from collections import Counter  # Conteo de frecuencias de elementos (tokens, palabras, etc.)
import os  # Manejo de rutas, archivos y operaciones del sistema de archivos
import math  # Funciones matemáticas avanzadas (logaritmos, potencias, trigonometría, etc.)

# Deep Learning con PyTorch
import torch  # Librería principal de tensores y operaciones en GPU/CPU
import torch.nn as nn  # Definición de capas y módulos de redes neuronales
import torch.nn.functional as F  # Funciones de activación y operaciones matemáticas de redes
from torch.utils.data import Dataset, random_split, DataLoader, Subset  # Utilidades para crear y dividir datasets, cargar lotes y trabajar con subconjuntos

# Entrenamiento estructurado con PyTorch Lightning
from pytorch_lightning import LightningModule, Trainer  # Clase base y manejador de entrenamiento de modelos
from pytorch_lightning.loggers import TensorBoardLogger  # Registro de métricas e historial en TensorBoard
from pytorch_lightning.callbacks.early_stopping import EarlyStopping  # Detener entrenamiento si no mejora la métrica
from torchmetrics import Accuracy  # Métrica de precisión para clasificación supervisada

# Tipado para mayor legibilidad y validación de funciones
from typing import Tuple, Dict, Optional  # Definición de tipos de datos para funciones y estructuras
from enum import Enum  # Definición de enumeraciones (conjuntos de valores constantes con nombre)

from tqdm.auto import tqdm  # Barra de progreso adaptable para bucles
from transformers import AutoTokenizer  # Tokenizador automático de modelos preentrenados de Hugging Face
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode  # Conversión de bytes a caracteres Unicode (usado en tokenización tipo GPT-2)

import optuna  # Optimización automática de hiperparámetros mediante búsquedas eficientes (Bayesian, TPE, etc.)

# Métricas de evaluación con Scikit-learn
from sklearn.model_selection import train_test_split  # División de datos en conjuntos de entrenamiento y prueba
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas de evaluación de modelos de clasificación

### 2. Cargue de datos

Este dataset contiene 2.2k conversaciones multi-turno generadas por Llama-3.1-70B-Instruct. Se le solicitó al modelo generar una conversación sencilla, con 3 a 4 intercambios breves, entre un Usuario y un Asistente de IA sobre un tema específico.

De este dataset se utilizarán específicamente las conversaciones entre el usuario y el asistente de IA, con el fin de asignarlas al tópico correspondiente. Cada ejemplo consiste en un diálogo corto multi-turno, donde el modelo debe identificar el tema principal a partir del intercambio entre ambas partes.

Está disponible en el Hugging Face Hub, lo que permite descargarlo fácilmente y utilizarlo directamente en entrenamientos o pruebas de modelos de NLP.

In [35]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
dataset = load_dataset("HuggingFaceTB/everyday-conversations-llama3.1-2k")
dataset

DatasetDict({
    train_sft: Dataset({
        features: ['topic', 'subtopic', 'subsubtopic', 'full_topic', 'prompt', 'completion', 'token_length', 'messages'],
        num_rows: 2260
    })
    test_sft: Dataset({
        features: ['topic', 'subtopic', 'subsubtopic', 'full_topic', 'prompt', 'completion', 'token_length', 'messages'],
        num_rows: 119
    })
})

El dataset incluye varias columnas pero me centraré en dos de ellas: La conversación entre el user y la AI (completion), la categoría a la que pertenece la conversación (topic). Para entrenar y validar el modelo, se dispone de 2.260 filas en el conjunto de entrenamiento y 119 en el conjunto de prueba.

Concatenos los dos datasets para tener mayor cantidad de datos.

In [36]:
tr_dataset = dataset["train_sft"]
te_dataset = dataset["test_sft"]
full_dataset = concatenate_datasets([tr_dataset, te_dataset])

Observemos uno de sus registros

In [37]:
full_dataset[0]

{'topic': 'Shopping',
 'subtopic': 'Budgeting',
 'subsubtopic': 'Tracking expenses',
 'full_topic': 'Shopping/Budgeting/Tracking expenses',
 'prompt': 'Generate a very simple multi-turn conversation between a User and an AI Assistant about Shopping/Budgeting/Tracking expenses. The conversation should start with a basic greeting like "Hello" or "Hi" and be straightforward. Include 3-4 short exchanges. The AI should give brief, clear answers. The User should ask simple questions.\n\nStart the conversation like this:\n\nUser: [Greeting]\n\nAI: Hello! How can I help you today?\n\nUser: [Continue with a simple question or statement]\n\nAI: [Respond briefly and clearly]\n\nUser: [Ask a follow-up question or make another simple statement]\n\nAI: [Provide a final helpful response]\n\nMake sure the entire conversation remains very simple and easy to understand, focusing on basic topics or requests.',
 'completion': "User: Hi\n\nAI: Hello! How can I help you today?\n\nUser: I'm trying to track m

Revisamos especificamente las 2 columnas de interes.

In [38]:
full_dataset = full_dataset.select_columns(["completion", "topic"])

Ajusto el nombre de las columnas a text y category.

In [39]:
full_dataset = full_dataset.rename_column("completion", "text")
full_dataset = full_dataset.rename_column("topic", "category")

In [40]:
dataset = full_dataset

Ahora observemos uno de los registros ya ajustado.

In [41]:
dataset[0]

{'text': "User: Hi\n\nAI: Hello! How can I help you today?\n\nUser: I'm trying to track my expenses. Can you help me with that?\n\nAI: Yes, I can help you track your expenses. You can start by telling me your income and fixed expenses, such as rent and utilities.\n\nUser: That sounds easy. How do I know if I'm staying within my budget?\n\nAI: Once you've entered your income and expenses, I can help you set a budget and alert you when you're going over. You can also categorize your spending to see where your money is going.\n\nUser: Okay, that sounds great. Can you remind me to review my budget regularly?\n\nAI: I can send you reminders to review your budget weekly, monthly, or at any interval you prefer.",
 'category': 'Shopping'}

In [42]:
pd.Series(dataset['category']).value_counts()

Cooking                  100
Travel                   100
Sports                   100
Health                   100
Pets                     100
Technology               100
Work                     100
Education                100
Weather                  100
Fashion                  100
Transportation           100
Hobbies                  100
Food                     100
Sleep                    100
Fitness                  100
Home                     100
Entertainment            100
Family                   100
Shopping                  99
Music                     50
communication             10
culture                   10
environmental science     10
plants                    10
food chains               10
everyday objects          10
simple machines           10
sound                     10
astronomy                 10
genetics and heredity     10
basic math concepts       10
time                      10
ecosystems                10
light                     10
animals       

Se encuentra que el dataset esta desbalanceado, pues algunas categorias tienes 100 registros pero otros 10.

Para efectos de la tarea solo utilizaremos las categorias que tienen una muestra suficiente y permite tener el dataset balanceado.

In [43]:
counts = Counter(dataset['category'])
dataset = dataset.filter(lambda example: counts[example['category']] > 98)

In [44]:
pd.Series(dataset['category']).value_counts()

Cooking           100
Travel            100
Work              100
Transportation    100
Technology        100
Pets              100
Health            100
Sports            100
Weather           100
Education         100
Fashion           100
Family            100
Sleep             100
Food              100
Hobbies           100
Entertainment     100
Fitness           100
Home              100
Shopping           99
Name: count, dtype: int64

In [45]:
len(set(dataset['category']))

19

In [46]:
dataset.shape

(1899, 2)

Finalmente quedamos con 19 categorias balanceadas y un total de 1899 registros.

A manera general, observemos que tan largos o cortos tienden a ser los textos.

In [47]:
text_lengths = [len(row['text']) for row in dataset]
print(f"Texto más corto: {min(text_lengths)}")
print(f"Texto más largo: {max(text_lengths)}")
print(f"Longitud promedio: {sum(text_lengths) / len(text_lengths)}")

Texto más corto: 270
Texto más largo: 1445
Longitud promedio: 747.9057398630858


Estos valores son la cantidad de *caractéres* que tiene las secuencias. Una decisión ingenua pero útil en este momento podría ser ajustar la longitud de las secuencias que vamos a usar para el entrenamiento a unos 350 tokens. Esto podría ser suficiente para capturar una porción significativa de los textos.

### 3. Definición del Tokenizer

Ahora, vamos a definir el tokenizer para nuestra tarea. Para ahorrarnos tiempo, vamos a entrenar uno basado en gpt2, pero ajustandolo a nuestro dataset. Para ello, debemos seleccionar una muestra representativa de nuestro dataset, como no es muy grande, casi que podemos usarlo todo. Luego, debemos definir el tamaño del vocabulario, es decir, cuantos tokens únicos queremos soportar en nuestro tokenizador. Para que un modelo de lenguaje funcione moderadamente bien para una tarea de clasificación, considerando el tamaño de nuestro corpus, deberíamos definir unos 50 mil tokens.

In [48]:
length = 350
iter_dataset = iter(dataset)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

def batch_iterator(batch_size: int = 10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['text'] for _ in range(batch_size)]

english_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=50000, initial_alphabet=base_vocab)

100%|██████████| 35/35 [00:00<00:00, 376.98it/s]


Exploremos ahora el tokenizador obtenido.

In [49]:
tokens = sorted(english_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print(f"Vocabulario: {english_tokenizer.vocab_size} tokens")
print("Primeros 15 tokens:")
print([f"{english_tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[:15]])
print("15 tokens de en medio:")
print([f"{english_tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[1000:1015]])
print("Últimos 15 tokens:")
print([f"{english_tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[-15:]])

Vocabulario: 8811 tokens
Primeros 15 tokens:
['<|endoftext|>', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.']
15 tokens de en medio:
[' might', 'ening', ' over', 'ish', ' hob', ' rem', 'ables', ' Ch', ' exper', ' mod', ' learn', ' avail', ' something', ' hobb', "'ve"]
Últimos 15 tokens:
[' lasagna', ' dynamics', ' Superhero', ' volunteering', ' deadheading', ' journalism', ' disagreements', ' Vincennes', 'Bridgerton', 'Cyberpunk', ' Queenstown', ' civilizations', ' pedestrian', ' Hedgehogs', ' inexperienced']


Vemos que los primeros tokens corresponden a caracteres especiales y puntiación. Luego en el medio tenemos una combinación entre palabras completas y cortadas, el tokenizador se encarga de encontrar las frecuencias más comunes y asi partir las palabras por aquellas partes que tienden a repetirse mas. Esto es muy útil para trabajar con modelos de lenguaje ya que el modelo se vuelve robusto a diferentes ramificaciones de palabras e incluso a errores de tipografía. Finalmente, al final, vemos que tenemos más palabras cortadas y palabras muy especiales.

### 4. Definición del dataset de pytorch

Ahora podemos proceder a definir el dataset.

In [50]:
class EnglishCatDataset(Dataset):

    def __init__(self, tokenizer, dataset, seq_length: int = 512):
        self.tokenizer = tokenizer
        self.tokenizer.pad_token = '[PAD]'
        self.dataset = dataset
        self.seq_length = seq_length
        self.id_2_class_map = dict(enumerate(np.unique(dataset[:]['category'])))
        self.class_2_id_map = {v: k for k, v in self.id_2_class_map.items()}
        self.num_classes = len(self.id_2_class_map)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        text, y = self.dataset[index]['text'], self.dataset[index]['category']
        y = self.class_2_id_map[y]
        data = {k: torch.tensor(v) for k, v in self.tokenizer(text, max_length=self.seq_length, truncation=True, padding='max_length').items()}
        data['y'] = torch.tensor(y)
        return data


    def __len__(self):
        return len(self.dataset)

Ahora instanciaremos el dataset entero. Para este experimento, definiremos un tamaño máximo de secuencia de 2048 **tokens**. Que según nuestra intuición arriba, debería ser suficiente para la tarea.

In [51]:
max_len = 350 
english_cat_dataset = EnglishCatDataset(english_tokenizer, dataset, seq_length=max_len)
assert len(english_cat_dataset) == len(dataset)

Y luego, procedemos a hacer el train-val-test split y crear los dataloaders. Se mantienen las proporciones de cada categoria en cada dataset.

In [52]:
batch_size = 4 if not IN_COLAB else 12
train_dataset, val_dataset, test_dataset = random_split(english_cat_dataset, lengths=[0.8, 0.1, 0.1])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

### Definición de los Positional Embeddings

Según el paper, los autores agregan una secuencia sinusoidal a los embeddings de los tokens con el fin de inyectar información referente a la posición de cada token en las frases. Esto obedece a la definición:

$$
PE(pos, 2i) = \sin(pos/10000^{2i/d_{model}}) \\
PE(pos, 2i + 1) = \cos(pos/10000^{2i/d_{model}})
$$

Donde: 
- $pos$ es la posición del *token* en la secuencia.
- $i$ es la dimensión $i$ en el embedding $d$.
- $d_model$ es la dimensionalidad total del embedding.

Lo que los autores propusieron fue que para las posiciones pares, se calculara el seno de la posición, relativa a la dimensionalidad del embedding y para las posiciones impares, se calculara el coseno. Según los autores, estos tenían la hipótesis de que estas funciones inyectarían la información posicional relativa de forma eficiente, en parte porque se pueden pre-calcular e inyectar directamente durante el entrenamiento, evitando asi emplear recursos en entrenar estructuras para aprenderlos.

Esto último es particularmente importante ya que se evita tanto hacer uso de recursos innecesarios como acelerar el proceso de entrenamiento al no tener que computar gradientes para esta parte. Sin embargo, los autores también mencionaron que es ciertamente posible aprender estos positional embeddings como parte del entrenamiento y que según sus resultados, no había mucha diferencia entre ambos enfoques, razón por la cual, se prefiere el positional encoding sinusoidal.

In [53]:
class PosEncodingType(Enum):
    SINUSOID = 1
    LEARNABLE = 2


class SinusoidPE(nn.Module):

    def __init__(self, max_len: int, d_model: int):
        super(SinusoidPE, self).__init__()
        
        # Definimos un vector columna con las posiciones de la secuencia de entrada (pos)
        pos = torch.arange(max_len).unsqueeze(1)
        # Definimos un vector de fila con las dimensiones del embedding (i)
        i = torch.arange(d_model).unsqueeze(0)

        # Calculamos el denominador segun la formula
        div_term = 1 / torch.pow(10000, (2 * (i // 2)) / torch.tensor(d_model, dtype=torch.float32))
        # Aplicamos el denominador a las posiciones
        angle_rads = pos * div_term

        # Inicializamos la matriz de positional encodings
        pos_encoding = torch.zeros(max_len, d_model)
        # Calculamos los embeddings para los numeros pares con seno: PE(pos, 2i)
        pos_encoding[:, 0::2] = torch.sin(angle_rads[:, 0::2])
        # Calculamos los embdeddings para los numeros inpares con coseno: PE(pos, 2i+1)
        pos_encoding[:, 1::2] = torch.cos(angle_rads[:, 1::2])
        
        # Registramos la variable como atributo de clase
        self.register_buffer("pos_encoding", pos_encoding.unsqueeze(0), persistent=False)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.pos_encoding[:, :x.size(1), :]
    

class LearnablePE(nn.Module):

    def __init__(self, vocab_size: int, d_model: int, max_len: int = float('-inf')):
        super(LearnablePE, self).__init__()
        self.max_len = max_len
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        positions = torch.arange(0, max(x.size(-1), self.max_len))
        pos_emb = self.embedding(positions)
        return x + pos_emb



class TokenAndPosEmbedding(nn.Module):

    def __init__(self, max_len: int, embed_dim: int, vocab_size: int, pos_encoding_type: PosEncodingType = PosEncodingType.SINUSOID):
        super(TokenAndPosEmbedding, self).__init__()
        self.token_emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        if pos_encoding_type == PosEncodingType.SINUSOID:
            self.pos_emb = SinusoidPE(max_len, embed_dim)
        else:
            self.pos_emb = LearnablePE(vocab_size, embed_dim)


    def forward(self, x):
        token_emb = self.token_emb(x)
        return self.pos_emb(token_emb)


Ahora procedemos a instanciar el modulo que va a convertir los tokens en embeddings con positional embeddings.

In [54]:
emb_dim = 128 if not IN_COLAB else 256
tpe = TokenAndPosEmbedding(max_len, emb_dim, english_tokenizer.vocab_size)
pos_encoding = tpe.pos_emb.pos_encoding.squeeze(0).numpy()

In [55]:
text = "hola mundo!"
tokens = english_tokenizer(text, max_length=max_len, truncation=True, padding='max_length')
x = torch.tensor(tokens['input_ids']).unsqueeze(0)
mask = torch.tensor(tokens['input_ids']).unsqueeze(0)
embedding = tpe(x)
embedding.shape

torch.Size([1, 350, 128])

### Multi-Head Attention

Ahora procedemos a definir al núcleo del modelo. Recodemos que la atención se define por:

$$
\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_K}})V
$$

Que es la definición de "Scaled Dot-Product Attention". Y Multi-Head Attention es la concatenación de varias cabezas ejecutando el mismo scaled dot-product sobre partes del input. Entonces tenemos:

In [56]:
class MultiHeadAttention(nn.Module):

    def __init__(self, embed_size: int, num_heads: int = 8):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        assert embed_size & num_heads == 0, 'El tamaño del embedding debería ser divisible por el numero de cabezas'
        self.projection_dim = embed_size // num_heads
        self.query = nn.Linear(emb_dim, emb_dim)
        self.key = nn.Linear(emb_dim, emb_dim)
        self.value = nn.Linear(emb_dim, emb_dim)
        self.comibe_heads = nn.Linear(emb_dim, emb_dim)


    @staticmethod
    def _scaled_dot_product(q, k, v, mask=None):
        # d_k para el escalamiento
        d_k = q.size()[-1]

        # multiplicacion Q \cdot K^T 
        attn_logits = torch.matmul(q, k.transpose(-2, -1))
        # escalamiento
        attn_logits = attn_logits / math.sqrt(d_k)
        
        # Se aplica la máscara
        if mask is not None:
            attn_logits = attn_logits.masked_fill(mask.reshape(mask.shape[0], 1, 1, -1) == 0, -9e-15)

        # Se calcula el score de atención.
        attention = torch.softmax(attn_logits, dim=-1)
        # Se obtienen los valores tras el score de atención.
        values = torch.matmul(attention, v)
        return values, attention
    

    def _separate_heads(self, x, batch_size):
        # Llega: (batch, seq_len, emb_dim)
        x =  x.reshape(batch_size, -1, self.num_heads, self.projection_dim)  # (batch, seq_len, num_heads, emb_dim / num_heads)
        return x.permute(0, 2, 1, 3)  # (batch, num_heads, seq_len, emb_dim / num_heads)
    

    def forward(self, x, mask=None, return_attention=False):
        # x: (batch, seq_len, emb_dim)
        batch_size, seq_len, emb_dim = x.size()
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        q = self._separate_heads(q, batch_size)
        k = self._separate_heads(k, batch_size)
        v = self._separate_heads(v, batch_size)

        weights, attention = self._scaled_dot_product(q, k, v, mask)
        weights = weights.permute(0, 2, 1, 3) # (batch, seq_len, num_heads, emb_dim / num_heads)
        weights = weights.reshape(batch_size, seq_len, emb_dim)
        output = self.comibe_heads(weights)
        
        if return_attention:
            return output, attention
        else:
            return output

Podemos hacer una prueba rápida de que las operaciones funcionan a nivel de matrices.

In [57]:
mha = MultiHeadAttention(emb_dim)
mha(embedding, mask).shape

torch.Size([1, 350, 128])

### Definición del bloque transformers

Finalmente, definimos el bloque de transformers. Recordemos que como esta es una tarea de clasificación, solamente necesitamos el encoder, por lo que esto es silamente la primera parte del diseño de arquitecura de red.

En esta capa, simplemente ponemos una capa densa adicional junto con las normalizaciones a nivel de capa.

In [58]:
class TransformerBlock(nn.Module):

    def __init__(self, emb_dim: int, num_heads: int = 8):
        super(TransformerBlock, self).__init__()
        self.mhatt = MultiHeadAttention(emb_dim, num_heads)
        self.mhatt_dropput = nn.Dropout(0.2)
        self.ffn = nn.Sequential(
            nn.Linear(emb_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, emb_dim)
        )
        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)


    def forward(self, x, mask=None):
        attn_output = self.mhatt(x, mask)
        attn_output = self.mhatt_dropput(attn_output)
        attn_output = self.layer_norm1(attn_output)
        ffn_out = self.ffn(attn_output)
        return self.layer_norm2(ffn_out)


Nuevamente, probamos rapidamente para asegurarnos que las capas operan correctamente.

In [59]:
tb = TransformerBlock(emb_dim)
tb(embedding, mask).shape

torch.Size([1, 350, 128])

In [60]:
num_heads = 4
vocab_size = english_tokenizer.vocab_size

token_embeddings = TokenAndPosEmbedding(max_len, emb_dim, vocab_size)
transformer = TransformerBlock(emb_dim, num_heads)
ff = nn.Sequential(
    nn.Flatten(),
    nn.Linear(max_len * emb_dim, english_cat_dataset.num_classes)
)

In [ ]:
it = iter(train_loader)
batch = next(it)
x, mask, y = batch['input_ids'], batch['attention_mask'], batch['y']

embeddings = token_embeddings(x)
assert embeddings.shape == (train_loader.batch_size, max_len, emb_dim)

attention = transformer(embeddings, mask)
attention.shape

In [ ]:
pred = ff(attention)
pred.shape

### 6. Definición del clasificador

Finalmente, definimos el modelo en si. Este modelo constará de 3 capas:

- La tokenización, tal como la definimos anteriormente.
- El transformer, que acabamos de definir.
- Una capa densa adicional que servirá como clasificador de aquello que nos entregue la capa del transformer.

Como este es un LightningModule, aquí definiremos el resto de funciones utilitarias para el entrenamiento de la tarea.

In [ ]:
class EnglishClassifier(LightningModule):

    def __init__(self, max_len: int, vocab_size: int, num_classes: int, emb_dim: int, num_heads: int = 8):
        super(SpanishNewsClassifier, self).__init__()
        self.num_classes = num_classes

        self.token_embeddings = TokenAndPosEmbedding(max_len, emb_dim, vocab_size)
        self.transformer = TransformerBlock(emb_dim, num_heads)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(max_len * emb_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)
        )

        self.train_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_acc = Accuracy(task='multiclass', num_classes=num_classes)


    def forward(self, x, mask=None):
        embeddings = self.token_embeddings(x)
        attention = self.transformer(embeddings, mask)
        return self.classifier(attention)
    

    def training_step(self, batch):
        x, mask, y = batch['input_ids'], batch['attention_mask'], batch['y']
        y_hat = self(x, mask)
        loss = F.cross_entropy(y_hat, y)
        self.train_acc(y_hat, y)
        self.log('train-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train-acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch):
        x, mask, y = batch['input_ids'], batch['attention_mask'], batch['y']
        y_hat = self(x, mask)
        loss = F.cross_entropy(y_hat, y)
        self.val_acc(y_hat, y)
        self.log('val-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val-acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch):
        x, mask, y = batch['input_ids'], batch['attention_mask'], batch['y']
        y_hat = self(x, mask)
        self.test_acc(y_hat, y)
        self.log('test-acc', self.test_acc, prog_bar=True, on_step=False, on_epoch=True)


    def predict_step(self, batch):
        x, mask = batch['input_ids'], batch['attention_mask']
        return self(x, mask)


    def configure_optimizers(self):
        optimizer =  torch.optim.AdamW(self.parameters(), lr=2e-5, weight_decay=1e-5)
        return optimizer
    


In [ ]:

model = EnglishClassifier(max_len=english_cat_dataset.seq_length, vocab_size=english_tokenizer.vocab_size, num_classes=english_cat_dataset.num_classes, emb_dim=emb_dim)

tb_logger = TensorBoardLogger('tb_logs', name='TransformersClassifier')
callbacks=[EarlyStopping(monitor='train-loss', patience=3, mode='min')]
trainer = Trainer(max_epochs=10, devices=1, logger=tb_logger, callbacks=callbacks, precision="16-mixed")

In [ ]:
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Observemos el proceso de entrenamiento

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir tb_logs/

### 7. Evaluación de desempeño del modelo

Realizamos la validación contra el conjunto de prueba.

In [ ]:
model.eval()
trainer.test(model, test_loader)

### 8. Resultados

Finalmente, vamos a hacer uso del modelo y ver que tan bueno es para la clasificación de titulares de noticias.

In [ ]:
predictions = trainer.predict(model, test_loader)
predictions = torch.cat(predictions, dim=0)
predictions = torch.argmax(predictions, dim=-1)
predictions = [spanish_news_dataset.id_2_class_map[pred] for pred in predictions.tolist()]

In [ ]:
test_indices = test_dataset.indices
df = pd.DataFrame(data={
    "texto": full_dataset[test_indices]['text'],
    "tokens": [tokenize_text(v) for v in full_dataset[test_indices]['text']],
    "categoría": full_dataset[test_indices]['label'],
    'predicción': predictions
}, index=test_indices)

id_2_token = {v: k for k, v in vocab.items()}

df['tokens_string'] = df.tokens.apply(lambda t: ' '.join([id_2_token[i] for i in t]))
df = df[["texto", "tokens", "tokens_string", "categoría", "predicción"]]
df.style.set_table_styles(
    [
        {'selector': 'td', 'props': [('word-wrap', 'break-word')]}
    ]
)
df.head(15)

In [ ]:
errors = df[df['categoría'] != df['predicción']]
errors.head(15)

Se observa que, cuando los títulos no contienen palabras clave que diferencien claramente la categoría, el modelo tiende a mostrar mayor incertidumbre en su clasificación.

In [ ]:
df['predicción'].unique()

### 9. Conclusiones

#### Eficacia del flujo de análisis

- Se implementó un pipeline de clasificación multiclase sobre títulos de noticias en español utilizando un modelo basado en LSTM.
- El bloque LSTM actuó como featurizer, extrayendo representaciones de las secuencias de entrada a partir de las cuales se realizaron las predicciones.

#### Rendimiento del modelo

- El modelo alcanzó un accuracy de 0.9, lo que indica un desempeño bueno para la tarea de clasificación.
- Este resultado refleja que, en su configuración actual y sin optimización de hiperparámetros, el modelo aún no logra capturar de manera suficiente las características del texto para diferenciar entre las clases.

#### Limitaciones observadas

- El tiempo de entrenamiento fue elevado, lo cual es consistente con la naturaleza secuencial de las LSTM, donde en cada paso temporal se deben calcular gradientes.
- El modelo se entrenó sin realizar ajustes de hiperparámetros (tasa de aprendizaje, tamaño de batch, número de capas, etc.), lo cual limita su potencial de rendimiento.
- La representación basada únicamente en tokens y padding puede ser insuficiente para capturar matices semánticos complejos.

#### Áreas de mejora

- Explorar técnicas de optimización de hiperparámetros y regularización para mejorar la capacidad predictiva.
- Incorporar embeddings preentrenados en español (por ejemplo, FastText o Word2Vec) para enriquecer la representación semántica.
- Evaluar arquitecturas más modernas como GRU, Transformers o modelos preentrenados que podrían ofrecer un mejor trade-off entre rendimiento y tiempo de cómputo.

#### Valor práctico

- Aunque el rendimiento inicial es intermedio, el experimento permite validar el pipeline y sentar bases para futuros ajustes.
- Este método ofrece un punto de partida funcional para experimentar con arquitecturas más sofisticadas y con un mejor ajuste de hiperparámetros, lo cual podría elevar significativamente la precisión del clasificador.

In [ ]:
import pkg_resources

libs = [
    "numpy",
    "pandas",
    "datasets",
    "torch",
    "pytorch-lightning",
    "torchmetrics",
    "tqdm",
    "transformers",
    "scikit-learn"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except Exception:
        print(f"{lib}")